# 데이터 y 값 불러와서 match 해야함 

In [132]:
import tensorflow
from keras import layers, Sequential, backend
from keras.layers import Dense,Flatten,Conv2D,AveragePooling2D,MaxPool2D
from keras import Input, Model

from math import sqrt
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import pandas as pd

In [133]:
# 각 좌표값에서 수치 풀러와서 저장하기
# List의 조정으로 원하는 STN 조정 = [춘천,서울,인천,청주,포항,대구,전주,부산,흑산도,제주,진주]
# STN_List = [(425,489),(443,455),(448,440),(494,476),(527,561),(535,529),(537,462),(578,546),(600,389),(664,436),(574,503)]
# STN_List = [춘천(425,489),서울(443,455),인천(448,440),청주(494,476),포항(527,561),대구(535,529),전주(537,462),부산(578,546),흑산도(600,389),제주(664,436),진주(574,503)]
STN_List = [(425,489),(443,455),(448,440),(494,476),(527,561),(535,529),(537,462),(578,546),(600,389),(664,436),(574,503)]

In [134]:
def ExtractImage(path):  # 이미지에 대한 정보 추출
    img_l=[]
    img_name=[]
    path= path+'*00.png'  # 이미지 타입고르기 : 시간조정 

    for filename in glob.iglob(path, recursive=True):
        temp_img = cv2.imread(filename)
        img_l.append(temp_img)
        filename=filename[-16:-6]  # y값과 일치시키기 위해 필요한 joinkey의 뒷자리만큼 자르기 
        img_name.append(filename)
    return img_l,img_name


def extractWeather(weather_path,img_name):  # weather data preprocess
    csvfile=pd.read_csv(weather_path,encoding='cp949') # utf-8  or  cp949
    filtered_csv=[]
    for i in list(range(0,len(img_name))): #img의 날짜를 str ->int 필요함
        temp=csvfile.loc[csvfile['일시']==int(img_name[i]),['기온(°C)','풍속(m/s)','습도(%)','현지기압(hPa)','month','hour','일사(MJ/m2)']]
        temp=temp.values.tolist()
        filtered_csv.append(temp)
    filtered_csv=np.reshape( filtered_csv,newshape=(-1,7)) # data type의 문제 때문에 columns의 수를 수동 조정
    filtered_csv=pd.DataFrame(filtered_csv,columns=['기온(°C)','풍속(m/s)','습도(%)','현지기압(hPa)','month','hour','일사(MJ/m2)'])
   
    x_weather=filtered_csv.drop(['일사(MJ/m2)'],1)
    y_rad=filtered_csv['일사(MJ/m2)']
    y_rad=y_rad.values

    return x_weather,y_rad


def imgtoRGB(img_l,STN_List): # image preprocessing
    data_set3=[]  
    for i in list(range(0,len(img_l))):       
        img=img_l[i]  # 사진의 길이만큼 data_set2 tensor를 저장 


        data_set2=[] # 두개 채널의 rgb tensor
        for k in STN_List :  # 한개의 이미지에 대하여 23개 관측소x2channel, RGB값 산출
            data_set=[]  # 각 사진에서 23 길이의 rgb를 list에 넣음
            # print(k)
            i,j=k
            # print(i,j)
            img_r=img[i-1:i+2,j-1:j+2] # 9pixel cutting
            # print(img2)
            data_set.append(img_r)
            #merge channel 
            data_set2.append(data_set)
        data_set2=np.array(data_set2).reshape(-1,3,3,3)
        data_set3.append(data_set2)
        
        
    x_rgb=data_set3   
    shape=(len(img_l)*len(STN_List) ,3,3,3) # length of img_l*leng of stn=42757
    x_rgb=np.asanyarray(x_rgb)
    x_rgb=x_rgb.reshape(shape)    
        
    return x_rgb

In [135]:
# make test file
path="D:/DB/2016new2/"
weather_path='C:/Users/taehee/Desktop/paper/2016년3.csv' #station을 줄이면 y값.csv파일도 고쳐줘야함,y값만 따로 정리할 필요가 있음 

img_l,img_name=ExtractImage(path)
x_weather,y_rad =extractWeather(weather_path,img_name)
x_rgb_train=imgtoRGB(img_l,STN_List)

In [136]:
# np.shape(x_rgb_val)

In [137]:
# make test file
path="D:/DB/2015new2/"
weather_path='C:/Users/taehee/Desktop/paper/2015년3.csv' #station을 줄이면 y값.csv파일도 고쳐줘야함,y값만 따로 정리할 필요가 있음 

img_l,img_name=ExtractImage(path)
x_weather_val ,y_rad_val =extractWeather(weather_path,img_name)
x_rgb_val=imgtoRGB(img_l,STN_List)

## image model

In [184]:
def build_model():
    image_input = Input(shape=(3,3,3),name='image')
    im_nn = layers.Conv2D(36,1,activation='sigmoid')(image_input)
    im_nn = layers.Conv2D(36,1,activation='sigmoid')(im_nn)
    im_nn = layers.Conv2D(36,1,activation='sigmoid')(im_nn)
    im_nn = layers.MaxPool2D(pool_size=(1,1))(im_nn)

    im_nn=Flatten()(im_nn)
    first_dense = Dense(1, )(im_nn)
    #######
    weather_input =Input(shape=(6,),name='weather') # shape의 모양도 column의 수에 맞게 조정
    w_nn = layers.Dense(24,activation='sigmoid')(weather_input)
    w_nn = layers.Dense(24,activation='sigmoid')(w_nn)
    w_nn = layers.Dense(1,activation='sigmoid')(w_nn)

    second_dense = Dense(1)(w_nn)
    ########
    concatenated = layers.concatenate([first_dense,second_dense],axis=-1)
    im_d = layers.Dense(24,activation='sigmoid')(concatenated)
    im_d = layers.Dense(1,activation='relu')(im_d)

    model = Model([image_input,weather_input ],im_d)
    model.compile(loss='mse',optimizer='rmsprop',metrics=['mse']) 
    #model.summary()
    return model

In [185]:
k = 5
num_val_samples = len(x_weather) // k
num_epochs = 100
all_mse_histories = []
for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data_rgb = x_rgb_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_data_w = x_weather[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_rad[i * num_val_samples: (i + 1) * num_val_samples]
    
    

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data_rgb = np.concatenate([x_rgb_train[:i * num_val_samples],x_rgb_train[(i + 1) * num_val_samples:]],axis=0)
    partial_train_data_w = np.concatenate([x_weather[:i * num_val_samples],x_weather[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets = np.concatenate([y_rad[:i * num_val_samples],y_rad[(i + 1) * num_val_samples:]],axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_model()
    # 모델 훈련(verbose=0 이므로 훈련 과정이 출력되지 않습니다)
    history=model.fit([partial_train_data_rgb,partial_train_data_w],partial_train_targets,batch_size=100,epochs=30,validation_data=([x_rgb_val,x_weather_val],y_rad_val))
    mae_history = history.history['val_loss']
    all_mse_histories.append(mae_history)


처리중인 폴드 # 0
Train on 34206 samples, validate on 4092 samples
Epoch 1/30
34206/34206 [==============================] - 16s 459us/step - loss: 0.8577 - mean_squared_error: 0.8577 - val_loss: 0.8702 - val_mean_squared_error: 0.8702
Epoch 2/30
34206/34206 [==============================] - 10s 291us/step - loss: 0.7041 - mean_squared_error: 0.7041 - val_loss: 0.4863 - val_mean_squared_error: 0.4863
Epoch 3/30
34206/34206 [==============================] - 10s 295us/step - loss: 0.5523 - mean_squared_error: 0.5523 - val_loss: 0.2507 - val_mean_squared_error: 0.2507
Epoch 4/30
34206/34206 [==============================] - 10s 282us/step - loss: 0.5256 - mean_squared_error: 0.5256 - val_loss: 0.2978 - val_mean_squared_error: 0.2978
Epoch 5/30
34206/34206 [==============================] - 9s 273us/step - loss: 0.5212 - mean_squared_error: 0.5212 - val_loss: 0.2734 - val_mean_squared_error: 0.2734
Epoch 6/30
34206/34206 [==============================] - 9s 269us/step - loss: 0.5158 - mean_s

Epoch 19/30
34206/34206 [==============================] - 10s 300us/step - loss: 0.4617 - mean_squared_error: 0.4617 - val_loss: 0.3155 - val_mean_squared_error: 0.3155
Epoch 20/30
34206/34206 [==============================] - 9s 267us/step - loss: 0.4599 - mean_squared_error: 0.4599 - val_loss: 0.2841 - val_mean_squared_error: 0.2841
Epoch 21/30
34206/34206 [==============================] - 10s 304us/step - loss: 0.4590 - mean_squared_error: 0.4590 - val_loss: 0.3762 - val_mean_squared_error: 0.3762
Epoch 22/30
34206/34206 [==============================] - 9s 267us/step - loss: 0.4594 - mean_squared_error: 0.4594 - val_loss: 0.3139 - val_mean_squared_error: 0.3139
Epoch 23/30
34206/34206 [==============================] - 10s 294us/step - loss: 0.4593 - mean_squared_error: 0.4593 - val_loss: 0.2799 - val_mean_squared_error: 0.2799
Epoch 24/30
34206/34206 [==============================] - 9s 262us/step - loss: 0.4575 - mean_squared_error: 0.4575 - val_loss: 0.3156 - val_mean_squar

34206/34206 [==============================] - 9s 254us/step - loss: 0.8029 - mean_squared_error: 0.8029 - val_loss: 0.7523 - val_mean_squared_error: 0.7523
Epoch 7/30
34206/34206 [==============================] - 9s 268us/step - loss: 0.8027 - mean_squared_error: 0.8027 - val_loss: 0.7070 - val_mean_squared_error: 0.7070
Epoch 8/30
34206/34206 [==============================] - 10s 307us/step - loss: 0.8020 - mean_squared_error: 0.8020 - val_loss: 0.7803 - val_mean_squared_error: 0.7803
Epoch 9/30
34206/34206 [==============================] - 10s 284us/step - loss: 0.8015 - mean_squared_error: 0.8015 - val_loss: 0.6876 - val_mean_squared_error: 0.6876
Epoch 10/30
34206/34206 [==============================] - 10s 282us/step - loss: 0.8013 - mean_squared_error: 0.8013 - val_loss: 0.8278 - val_mean_squared_error: 0.8278
Epoch 11/30
34206/34206 [==============================] - 11s 309us/step - loss: 0.8013 - mean_squared_error: 0.8013 - val_loss: 0.5843 - val_mean_squared_error: 0.58

34206/34206 [==============================] - 10s 293us/step - loss: 2.4330 - mean_squared_error: 2.4330 - val_loss: 0.6547 - val_mean_squared_error: 0.6547
Epoch 25/30
34206/34206 [==============================] - 10s 296us/step - loss: 2.4330 - mean_squared_error: 2.4330 - val_loss: 0.6547 - val_mean_squared_error: 0.6547
Epoch 26/30
34206/34206 [==============================] - 11s 313us/step - loss: 2.4330 - mean_squared_error: 2.4330 - val_loss: 0.6547 - val_mean_squared_error: 0.6547
Epoch 27/30
34206/34206 [==============================] - 10s 283us/step - loss: 2.4330 - mean_squared_error: 2.4330 - val_loss: 0.6547 - val_mean_squared_error: 0.6547
Epoch 28/30
34206/34206 [==============================] - 10s 296us/step - loss: 2.4330 - mean_squared_error: 2.4330 - val_loss: 0.6547 - val_mean_squared_error: 0.6547
Epoch 29/30
34206/34206 [==============================] - 10s 307us/step - loss: 2.4330 - mean_squared_error: 2.4330 - val_loss: 0.6547 - val_mean_squared_error:

In [186]:
num_epochs =30
average_mae_history = [np.sqrt(np.mean([x[i] for x in all_mse_histories])) for i in range(num_epochs)]
average_mae_history

[0.7974401757654697,
 0.7382089437455065,
 0.711279571507615,
 0.6843486942845756,
 0.675792736296002,
 0.7110507338888089,
 0.7204032478140692,
 0.7083215631158317,
 0.7004441004072716,
 0.7478263506638638,
 0.6750467375819701,
 0.6765571161902877,
 0.7139621345295512,
 0.6679707032672454,
 0.7160543981800398,
 0.7246357976970601,
 0.6774036864444932,
 0.7177204923293895,
 0.7226647945861471,
 0.7001480537328605,
 0.6985856065049098,
 0.6586599127941937,
 0.656304023193343,
 0.7051600764954348,
 0.6791114919967566,
 0.6741059318444309,
 0.7002749445825611,
 0.6822932421556264,
 0.6869659501872222,
 0.6799867930705947]

In [187]:
mse_history = history.history['val_loss']
np.mean(np.sqrt(mse_history))

0.8091603384496372

## 원본

In [188]:
def paper_model(x_rgb_train,y_rad,x_rgb_val,y_rad_val):
    model = Sequential()
    model.add(Conv2D(72,1,activation='sigmoid',input_shape=(3,3,3),strides=2))
    model.add(Conv2D(36,1,activation='sigmoid',input_shape=(3,3,3)))
    model.add(Conv2D(36,1,activation='sigmoid',input_shape=(3,3,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    
    model.add(Flatten())
    model.add(Dense(36,activation='sigmoid')) # 변경가능
    model.add(Dense(24,activation='sigmoid')) # 변경가능

    model.add(Dense(1))
    model.compile(loss='mse',optimizer='rmsprop',metrics=['mse'])   
    model.summary()
    # model.save('paper.h5')


    # -----
    hist=model.fit(x_rgb_train,y_rad,batch_size=100,epochs=20,validation_data=(x_rgb_val,y_rad_val) ) 
    #hist=model.fit(x_train,y_train,batch_size=1000,epochs=30 )
    # predict부분 구현해야 함   
    return hist

In [189]:
hist=paper_model(x_rgb_train,y_rad,x_rgb_val,y_rad_val)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_205 (Conv2D)          (None, 2, 2, 72)          288       
_________________________________________________________________
conv2d_206 (Conv2D)          (None, 2, 2, 36)          2628      
_________________________________________________________________
conv2d_207 (Conv2D)          (None, 2, 2, 36)          1332      
_________________________________________________________________
max_pooling2d_96 (MaxPooling (None, 1, 1, 36)          0         
_________________________________________________________________
flatten_96 (Flatten)         (None, 36)                0         
_________________________________________________________________
dense_602 (Dense)            (None, 36)                1332      
_________________________________________________________________
dense_603 (Dense)            (None, 24)                888       
__________

In [190]:
mse_history = hist.history['val_loss']
np.mean(np.sqrt(mse_history))

0.7909158461246398

In [155]:
# 월구분,stn구분은 해서 비교표를 결과로 넣을 것

In [172]:
def paper_model(x_rgb_train,y_rad,x_rgb_val,y_rad_val):
    model = Sequential()
    model.add(Conv2D(72,1,activation='sigmoid',input_shape=(3,3,3),strides=2))
    model.add(Conv2D(36,1,activation='sigmoid',input_shape=(3,3,3)))
    model.add(Conv2D(36,1,activation='sigmoid',input_shape=(3,3,3)))
    model.add(MaxPool2D(pool_size=(2,2)))
    
    
    model.add(Flatten())
    model.add(Dense(36,activation='sigmoid')) # 변경가능
    model.add(Dense(24,activation='sigmoid')) # 변경가능

    model.add(Dense(1))
    model.compile(loss='mse',optimizer='adam',metrics=['mse'])   
    model.summary()
    # model.save('paper.h5')


    # -----
    hist=model.fit(x_rgb_train,y_rad,batch_size=100,epochs=20,validation_data=(x_rgb_val,y_rad_val) ) 
    #hist=model.fit(x_train,y_train,batch_size=1000,epochs=30 )
    # predict부분 구현해야 함   
    return hist

In [174]:
hist=paper_model(x_rgb_train,y_rad,x_rgb_val,y_rad_val)
mse_history = hist.history['val_loss']
np.mean(np.sqrt(mse_history))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_178 (Conv2D)          (None, 2, 2, 72)          288       
_________________________________________________________________
conv2d_179 (Conv2D)          (None, 2, 2, 36)          2628      
_________________________________________________________________
conv2d_180 (Conv2D)          (None, 2, 2, 36)          1332      
_________________________________________________________________
max_pooling2d_87 (MaxPooling (None, 1, 1, 36)          0         
_________________________________________________________________
flatten_87 (Flatten)         (None, 36)                0         
_________________________________________________________________
dense_547 (Dense)            (None, 36)                1332      
_________________________________________________________________
dense_548 (Dense)            (None, 24)                888       
__________

0.7869787169201123

In [ ]:
 # 정확도를 위해서는 월별로, 지점별로 테스트 해봐야할것 같다.

hist_dict=hist.history
hist_dict.keys()

val_loss= hist_dict['val_loss']
loss = hist_dict['loss']
epochs =range(1,len(loss)+1)

plt.plot(epochs,loss,'bo-',label='Training loss')
plt.plot(epochs,val_loss,'r^-',label='val loss')

plt.title('loss graph, Nov, 18h')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

plt.show()